# Question 2b0

Prepare

# Imports and Setup

In [ ]:
from google.colab import drive
drive.mount('/gdrive', force_remount=True)

# chuanxin
%cd "../gdrive/My Drive/cz4042_assignment_2/2b" 

Mounted at /gdrive
/gdrive/My Drive/cz4042_assignment_2/2a


In [ ]:
import os
import pickle
import json
import csv
import re
# import pylab

import matplotlib.pyplot as plt
%matplotlib inline

import numpy as np
import tensorflow as tf
from tqdm.keras import TqdmCallback



In [ ]:
seed = 0
np.random.seed(seed)
tf.random.set_seed(seed)

inspect_index = 40

# Helper functions

### read_data_chars()
Used to load in the data. Returns x_train, y_train, x_test, y_test

In [ ]:
def read_data_chars():
    x_train, y_train, x_test, y_test = [], [], [], []
    cop = re.compile("[^a-z^A-Z^0-9^,^.^' ']")
    with open('./train_medium.csv', encoding='utf-8') as filex:
        reader = csv.reader(filex)
        for row in reader:
            data = cop.sub("", row[1])
            x_train.append(data)
            y_train.append(int(row[0]))

    with open('./test_medium.csv', encoding='utf-8') as filex:
        reader = csv.reader(filex)
        for row in reader:
            data = cop.sub("", row[1])
            x_test.append(data)
            y_test.append(int(row[0]))


    vocab_size, char_to_ix = vocabulary(x_train+x_test)
    x_train = preprocess(x_train, char_to_ix, MAX_DOCUMENT_LENGTH)
    y_train = np.array(y_train)
    x_test = preprocess(x_test, char_to_ix, MAX_DOCUMENT_LENGTH)
    y_test = np.array(y_test)

    x_train = tf.constant(x_train, dtype=tf.int64)
    y_train = tf.constant(y_train, dtype=tf.int64)
    x_test = tf.constant(x_test, dtype=tf.int64)
    y_test = tf.constant(y_test, dtype=tf.int64)

    return x_train, y_train, x_test, y_test

### vocabulary(strings)
Read data with [character]. Get the unique characters in this strings and the vocab size (of characters)

In [ ]:
def vocabulary(strings):
    chars = sorted(list(set(list(''.join(strings)))))
    char_to_ix = { ch:i for i,ch in enumerate(chars) }
    vocab_size = len(chars)
    return vocab_size, char_to_ix

### preprocess(strings, char_to_ix, MAX_LENGTH)
Clean up a string 

In [ ]:
def preprocess(strings, char_to_ix, MAX_LENGTH):
    data_chars = [list(d.lower()) for _, d in enumerate(strings)]
    for i, d in enumerate(data_chars):
        if len(d)>MAX_LENGTH:
            d = d[:MAX_LENGTH]
        elif len(d) < MAX_LENGTH:
            d += [' '] * (MAX_LENGTH - len(d))
            
    data_ids = np.zeros([len(data_chars), MAX_LENGTH], dtype=np.int64)
    for i in range(len(data_chars)):
        for j in range(MAX_LENGTH):
            data_ids[i, j] = char_to_ix[data_chars[i][j]]
    return np.array(data_ids)

### make_directories()
Used to create directories that might not have been made

In [ ]:
# Create folder to store histories and figures
def make_directories():
  if not os.path.exists('./histories'):
      os.mkdir('./histories')
  if not os.path.exists('./figures'):
      os.mkdir('./figures')    

### history_saver(history, filename, already_json=False)
Used to save a history object

In [ ]:
# filename like 'history/model_name.json'
def history_saver(history, model_name, already_json=False):
  history_json = {}

  if already_json:
    history_json = history
  else:
    history = history.history
    for key in history.keys():
      history_json[key] = history[key]

  with open('./histories/' + model_name, 'w') as file:
    json.dump(history_json, file)

  print("History saved")

### history_loader(filename)
Used to load in a json history object

In [ ]:
# filename like 'history/model_name.json'
def history_loader(model_name):
  with open('./histories/'+model_name) as json_file:
    history = json.load(json_file)
  print('History loaded')
  
  return history 

### plot_3_channel_img(np_array):

In [ ]:
def plot_3_channel_img(np_array, inspect_index=None, filename=None):
  if inspect_index == None:
    inspect_index = np.random.randint(len(np_array))

  titles = ['Original', 'Red channel', 'Green channel', 'Blue channel']
  cmaps = [None, plt.cm.Reds_r, plt.cm.Greens_r, plt.cm.Blues_r]

  fig, axes = plt.subplots(1, 4, figsize=(32,8))
  objs = zip(axes, (np_array[inspect_index], *np_array[inspect_index].transpose(2,0,1)), titles, cmaps)

  if filename == None:
    print("index", inspect_index)

  for ax, channel, title, cmap in objs:
    ax.imshow(channel, cmap=cmap)
    ax.set_title(title)
    ax.set_xticks(())
    ax.set_yticks(())
  
  if filename != None:
    fig.savefig(f'./figures/{filename}')

### plot_loss(history_json, model_name)
Plot out loss graph, and also save it

In [ ]:
def plot_loss(history_json, model_name):
  train_loss = history_json['loss']
  test_loss = history_json['val_loss']
  title = 'Model name: ' + model_name + '\nloss against epochs'

  plt.plot(train_loss, label='train')
  plt.plot(test_loss, label='test')
  plt.title(title)
  plt.ylabel('loss')
  plt.xlabel('epochs')
  plt.legend()
  plt.savefig(f'./figures/{model_name}_loss.png')
  
  plt.show()

### plot_acc(history_json, model_name)
Plot out accuracy graph, and also save it

In [ ]:
def plot_acc(history_json, model_name):
  train_acc = history_json['accuracy']
  test_acc = history_json['val_accuracy']
  title = 'Model name: ' + model_name + '\naccuracy against epochs'

  plt.plot(train_acc, label='train')
  plt.plot(test_acc, label='test')
  plt.title(title)
  plt.ylabel('accuracy')
  plt.xlabel('epochs')
  plt.legend()
  plt.savefig(f'./figures/{model_name}_accuracy.png')
  
  plt.show()

### plot_activations(model, model_name, x_data, x_data_name)
Get the intermediate activations for a single x_data (one item in x_train or x_test)

In [ ]:
def plot_activations(model, model_name, x_data, x_data_name):
  plot_3_channel_img(x_data, filename=f'{model_name}_{x_data_name}_source.png')

  layer_outputs = [layer.output for layer in model.layers]
  activation_model = tf.keras.Model(inputs=model.input, outputs=layer_outputs) # Creates a model that will return these outputs, given the model input
  activations = activation_model.predict(x_data) 

  activation_layer_names = ['conv1', 'max1', 'conv2', 'max2']  
  activations = activations[:4]

  images_per_row = 10
  
  for layer_name, layer_activation in zip(activation_layer_names, activations): # Displays the feature maps
    n_features = layer_activation.shape[-1] # Number of features in the feature map
    size = layer_activation.shape[1] #The feature map has shape (1, size, size, n_features).
    n_cols = n_features // images_per_row # Tiles the activation channels in this matrix
    display_grid = np.zeros((size * n_cols, images_per_row * size))

    for col in range(n_cols): # Tiles each filter into a big horizontal grid
      for row in range(images_per_row):
        channel_image = layer_activation[0, :, :, col * images_per_row + row]
        channel_image -= channel_image.mean() # Post-processes the feature to make it visually palatable
        channel_image /= channel_image.std()
        channel_image *= 64
        channel_image += 128
        channel_image = np.clip(channel_image, 0, 255).astype('uint8')
        display_grid[col * size : (col + 1) * size, row * size : (row + 1) * size] = channel_image # Displays the grid 
        
    scale = 1. / size
    plt.figure(figsize=(scale * display_grid.shape[1],
                        scale * display_grid.shape[0]))
    plt.title(layer_name)
    plt.grid(False)
    plt.axis('off')
    plt.imshow(display_grid, aspect='auto', cmap='viridis')
    plt.savefig(f'./figures/{model_name}_{x_data_name}_{layer_name}.png')
    plt.show()

# 2a0

In [ ]:
# # Training and test
# make_directories()
# x_train, y_train = load_data('data_batch_1')
# x_test, y_test = load_data('test_batch_trim')

### make_model(num_ch_c1, num_ch_c2, use_dropout)
Creates a CNN model that has number of channels for the Conv2D layer specified, and also dropout, if needed

In [ ]:
# def make_model(num_ch_c1, num_ch_c2, use_dropout):
#     model = tf.keras.Sequential()
#     model.add(tf.keras.layers.Input(shape=(32, 32, 3), name='input'))
#     model.add(tf.keras.layers.Conv2D(num_ch_c1, 9, activation='relu', input_shape=(None, None, 3), padding='valid', name='conv1'))
#     model.add(tf.keras.layers.MaxPool2D(pool_size=(2,2), strides=2, padding='valid', name='max1'))
#     model.add(tf.keras.layers.Conv2D(num_ch_c2, 5, activation='relu', input_shape=(None, None, 3), padding='valid', name='conv2'))
#     model.add(tf.keras.layers.MaxPool2D(pool_size=(2,2), strides=2, padding='valid', name='max2'))
#     model.add(tf.keras.layers.Flatten(name='flatten'))
#     if use_dropout:
#       model.add(tf.keras.layers.Dropout(rate=0.5, name='dropout1'))
#     model.add(tf.keras.layers.Dense(300, use_bias=True, name='dense1'))
#     if use_dropout:
#       model.add(tf.keras.layers.Dropout(rate=0.5, name='dropout2'))
#     model.add(tf.keras.layers.Dense(10, use_bias=True, input_shape=(300,), name='dense2'))  # Here no softmax because we have combined it with the loss    
    
#     return model

### set_model_parameters(optimizer_, model_name)
Select the optimizer and give the model a name that will be used to save it.

Returns `epochs, batch_size, loss, callbacks, optimizer, model_name`

In [ ]:
# def set_model_parameters(optimizer_, model_name):
#   # fixed 
#   epochs = 1000  
#   batch_size = 128  
#   learning_rate = 0.001 
#   loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
#   callbacks = [TqdmCallback(verbose=1)]


#   if optimizer_ == 'SGD':
#     optimizer = tf.keras.optimizers.SGD(learning_rate=learning_rate)
#   elif optimizer_ == 'SGD-momentum':
#     optimizer = tf.keras.optimizers.SGD(learning_rate=learning_rate, momentum=0.1)
#   elif optimizer_ == 'RMSProp': 
#     optimizer = tf.keras.optimizers.RMSprop(learning_rate=learning_rate)
#   elif optimizer_ == 'Adam': 
#     optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
#   else:
#     raise NotImplementedError(f'You do not need to handle [{optimizer_}] in this project.')

#   return epochs, batch_size, loss, callbacks, optimizer, model_name

### Hyperparameters

In [ ]:
# num_ch_c1_list = [10,30,50,70,90]
# num_ch_c2_list = [20,40,60,80,100]

In [ ]:
# for num_ch_c1 in num_ch_c1_list:
#   for num_ch_c2 in num_ch_c2_list:

#     model = make_model(num_ch_c1, num_ch_c2, False)
#     epochs, batch_size, loss, callbacks, optimizer, model_name = set_model_parameters('SGD', f'q2_model_{num_ch_c1}_{num_ch_c2}')

#     model.compile(optimizer=optimizer, loss=loss, metrics='accuracy')

#     history = model.fit(
#         x_train,
#         y_train,
#         batch_size=batch_size,
#         epochs=epochs,
#         callbacks=callbacks,
#         validation_data=(x_test, y_test))

#     history_saver(history, model_name)
#     history_json = history_loader(model_name)

#     plot_acc(history_json, model_name)
#     plot_loss(history_json, model_name)

#     for i in range(2):
#       print("-"*100)
#       print("x_test", i)
#       x_data = np.expand_dims(x_test[i], axis=0) 
#       plot_activations(model, model_name, x_data[:2], 'xtest' + str(i))
#       print("-"*100)